<a href="https://colab.research.google.com/github/dnrso/dnrso.github.io/blob/main/koBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install transformers
!pip install pytorch-lightning 
!pip install transformers 
# ! pip install --quiet "ipython[notebook]" "torch>=1.6, <1.9" "pytorch-lightning>=1.4" "torchmetrics>=0.6" "torchvision"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 708 kB 15.6 MB/s 
     |████████████████████████████████| 419 kB 50.7 MB/s 
     |████████████████████████████████| 5.9 MB 45.6 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 36.0 MB/s 
     |████████████████████████████████| 6.6 MB 56.4 MB/s 
     |████████████████████████████████

In [ ]:
%cd /content/drive/MyDrive/Github/KoBART-summarization

/content/drive/MyDrive/Github/KoBART-summarization


In [ ]:
# !git clone https://github.com/seujung/KoBART-summarization.git
git_path = "/content/drive/MyDrive/Github/KoBART-summarization/"

In [ ]:
!ls

data		     imgs		LICENSE      requirements.txt  train.py
dataset.py	     infer.py		logs	     rouge_metric.py
download_binary.py   install_kobart.sh	__pycache__  run_train_cpu.sh
get_model_binary.py  kobart_summary	README.md    run_train.sh


In [ ]:
import os
import pandas as pd
import sys
# sys.path.append("/content/drive/MyDrive/Github/KoBART-summarization")
# /content/drive/MyDrive/Github/KoBART-summarization/data/NaverReview.xlsx  네이버 헤드셋 리뷰
DATA_PATH = os.path.join(git_path, "data/") 
print("파일 크기 : ")
for file in os.listdir(DATA_PATH):
    if 'tsv' in file :
        print(file.ljust(30) + str(round(os.path.getsize(DATA_PATH + file) / 1000000, 2)) + 'MB')

파일 크기 : 
train.tsv                     89.64MB
test.tsv                      22.25MB
NaverReview.tsv               0.49MB
NaverReview_test.tsv          0.12MB


In [ ]:
# data = pd.read_csv(os.path.join(DATA_PATH,"train.tsv"), encoding='utf-8', sep='\t')
data = pd.read_excel(os.path.join(DATA_PATH,"NaverReview.xlsx"))
# data = pd.read_csv(os.path.join(DATA_PATH,"NaverReview.tsv"), encoding='utf-8', sep='\t')

In [ ]:
data[-4:]

,Unnamed: 0,Score,Review,Summary
2196,2196,4,"광고도 워낙 많이 하고, 드라마 예능에도 많이 나오는 멀티밤이 너무 궁금해서 반신반...",바르기 편합니다. 촉촉하고 윤기가 납니다. 주름이 옅어지는 것같아 꽤 만족스럽니다....
2197,2197,4,목주름 때문에 홈쇼핑 보다가 주문했네요가히 요새 광고도 많이 해서 익숙한 제품인데주...,냄새는 무향에 가깝습니다. 제형은 림밤같아요. 바르면 광채가 납니다. 근데 흡수력은...
2198,2198,4,기대가 너무 컷을까요? 촉촉하면서 흡수가 쏙 되길 바랫는데 머라카락 자꾸 달라붙고 ...,머리카락이 달라붙습니다. 유분기가 많아요. 스틱이라 사용이 편합니다. 매일 저녁 바...
2199,2199,4,가히멀티밤은 이젠 모르는 사람보다 아는사람이 더 많을듯~~~개인적으로 저는 향이 가...,향이 가볍습니다. 보습력이 좋아요. 가격대비 용량이 작습니다.


In [ ]:
data.head()

,Unnamed: 0,Score,Review,Summary
0,0,1,싼마이 듣는거 아주 좋음사실 대두는 절대 쓰면 안됨일반 사이즈의 머리라고 스스로 생...,대두는 크기가 맞지 않는다. 마이크 품질이 매우 안좋다.
1,1,1,학원에 아이들 온라인학습으로 7개 샀는데 첨에 마이크 녹음이 잘 안되어 전화드리니 ...,마이크 품질이 좋지 않아 녹음이 되지 않는다.
2,2,1,누구나 편안하게 사용 가능한 밴드조절 기능을 믿고 구매했는데 최대로 늘려도 작습니다...,머리가 커서 맞지 않는다. 소음 차단이 약하다.
3,3,1,헤드셋의 볼륨 조절기가 있는 장치의 옆에 있는 ON/OFF 스위치가 전혀 기능을 ...,전원 스위치가 작동을 안합니다.
4,4,1,마이크 문제 있습니다 개봉하고 두번 사용했는데 두번다 음성인식을 했다못했다 하네요 ...,마이크 품질이 좋지 않아 음성인식을 못한다.


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2200 non-null   int64 
 1   Score       2200 non-null   int64 
 2   Review      2200 non-null   object
 3   Summary     1025 non-null   object
dtypes: int64(2), object(2)
memory usage: 68.9+ KB


In [ ]:
data = data[['Review','Summary']]
data = data.dropna(axis = 0)  #결측치 제거
data.columns = ['news','summary']   #열이름 변경
data.head() 

,news,summary
0,싼마이 듣는거 아주 좋음사실 대두는 절대 쓰면 안됨일반 사이즈의 머리라고 스스로 생...,대두는 크기가 맞지 않는다. 마이크 품질이 매우 안좋다.
1,학원에 아이들 온라인학습으로 7개 샀는데 첨에 마이크 녹음이 잘 안되어 전화드리니 ...,마이크 품질이 좋지 않아 녹음이 되지 않는다.
2,누구나 편안하게 사용 가능한 밴드조절 기능을 믿고 구매했는데 최대로 늘려도 작습니다...,머리가 커서 맞지 않는다. 소음 차단이 약하다.
3,헤드셋의 볼륨 조절기가 있는 장치의 옆에 있는 ON/OFF 스위치가 전혀 기능을 ...,전원 스위치가 작동을 안합니다.
4,마이크 문제 있습니다 개봉하고 두번 사용했는데 두번다 음성인식을 했다못했다 하네요 ...,마이크 품질이 좋지 않아 음성인식을 못한다.


In [ ]:
#나누기전 리뷰 요약 데이터 저장
data.to_excel(os.path.join(DATA_PATH,"리뷰요약.xlsx"))

In [ ]:
data = pd.read_excel(os.path.join(DATA_PATH,"리뷰요약.xlsx"))

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1025 non-null   int64 
 1   news        1025 non-null   object
 2   summary     1025 non-null   object
dtypes: int64(1), object(2)
memory usage: 24.1+ KB


In [ ]:
from sklearn.model_selection import train_test_split
train_df , test_df = train_test_split(data , test_size = 0.2 , random_state=42)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 820 entries, 835 to 860
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  820 non-null    int64 
 1   news        820 non-null    object
 2   summary     820 non-null    object
dtypes: int64(1), object(2)
memory usage: 25.6+ KB


In [ ]:
train_df.to_csv('/content/drive/MyDrive/Github/KoBART-summarization/data/NaverReview.tsv', index=False, sep="\t")
test_df.to_csv('/content/drive/MyDrive/Github/KoBART-summarization/data/NaverReview_test.tsv', index=False, sep="\t")

In [ ]:
import argparse
import logging
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
from dataset import KobartSummaryModule
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
parser = argparse.ArgumentParser(description='KoBART Summarization')

parser.add_argument('--checkpoint_path',
                    type=str,
                    help='checkpoint path')


logger = logging.getLogger()
logger.setLevel(logging.INFO)
class ArgsBase():
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        parser.add_argument('--train_file',
                            type=str,
                            default='/content/drive/MyDrive/Github/KoBART-summarization/data/NaverReview.tsv',
                            help='train file')

        parser.add_argument('--test_file',
                            type=str,
                            default='/content/drive/MyDrive/Github/KoBART-summarization/data/NaverReview_test.tsv',
                            help='test file')

        parser.add_argument('--batch_size',
                            type=int,
                            default=14,
                            help='')
        parser.add_argument('--max_len',
                            type=int,
                            default=512,
                            help='max seq len')
        return parser

class Base(pl.LightningModule):
    def __init__(self, hparams, trainer, **kwargs) -> None:
        super(Base, self).__init__()
        self.save_hyperparameters(hparams)
        self.trainer = trainer

    @staticmethod
    def add_model_specific_args(parent_parser):
        # add model specific args
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)

        parser.add_argument('--batch-size',
                            type=int,
                            default=14,
                            help='batch size for training (default: 96)')

        parser.add_argument('--lr',
                            type=float,
                            default=3e-5,
                            help='The initial learning rate')

        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.1,
                            help='warmup ratio')

        parser.add_argument('--model_path',
                            type=str,
                            default=None,
                            help='kobart model path')
        return parser
    
    def setup_steps(self, stage=None):
        # NOTE There is a problem that len(train_loader) does not work.
        # After updating to 1.5.2, NotImplementedError: `train_dataloader` · Discussion #10652 · PyTorchLightning/pytorch-lightning https://github.com/PyTorchLightning/pytorch-lightning/discussions/10652
        train_loader = self.trainer._data_connector._train_dataloader_source.dataloader()
        return len(train_loader)

    def configure_optimizers(self):
        # Prepare optimizer
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(
                nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparams.lr, correct_bias=False)
        num_workers = self.hparams.num_workers

        data_len = self.setup_steps(self)
        logging.info(f'number of workers {num_workers}, data length {data_len}')
        num_train_steps = int(data_len / (self.hparams.batch_size * num_workers) * self.hparams.max_epochs)
        logging.info(f'num_train_steps : {num_train_steps}')
        num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
        logging.info(f'num_warmup_steps : {num_warmup_steps}')
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
        lr_scheduler = {'scheduler': scheduler, 
                        'monitor': 'loss', 'interval': 'step',
                        'frequency': 1}
        return [optimizer], [lr_scheduler]


class KoBARTConditionalGeneration(Base):
    def __init__(self, hparams, trainer=None, **kwargs):
        super(KoBARTConditionalGeneration, self).__init__(hparams, trainer, **kwargs)
        self.model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
        self.model.train()   # 학습 모드 설정
        self.bos_token = '<s>'
        self.eos_token = '</s>'
        
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):

        attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        decoder_attention_mask = inputs['decoder_input_ids'].ne(self.pad_token_id).float()
        
        return self.model(input_ids=inputs['input_ids'],
                          attention_mask=attention_mask,
                          decoder_input_ids=inputs['decoder_input_ids'],
                          decoder_attention_mask=decoder_attention_mask,
                          labels=inputs['labels'], return_dict=True)


    def training_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs.loss
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs['loss']
        return (loss)

    def validation_epoch_end(self, outputs):
        losses = []
        for loss in outputs:
            losses.append(loss)
        self.log('val_loss', torch.stack(losses).mean(), prog_bar=True)

if __name__ == '__main__':
    parser = Base.add_model_specific_args(parser)
    parser = ArgsBase.add_model_specific_args(parser)
    parser = KobartSummaryModule.add_model_specific_args(parser)
    parser = pl.Trainer.add_argparse_args(parser)
    tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
    args = parser.parse_args("--gradient_clip_val 1.0 --max_epochs 10 --default_root_dir logs --gpus 1 --batch_size 8 --num_workers 4".split())  #실행 인수
    logging.info(args)

    dm = KobartSummaryModule(args.train_file,
                        args.test_file,
                        tokenizer,
                        batch_size=args.batch_size,
                        max_len=args.max_len,
                        num_workers=args.num_workers)
    #체크 포인트 가장 작은 로스
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                                       dirpath=args.default_root_dir,
                                                       filename='model_chp/{epoch:02d}-{val_loss:.3f}',
                                                       verbose=True,
                                                       save_last=True,
                                                       mode='min',
                                                       save_top_k=3)
    #결과 보여주는 TensorBoardLogger
    tb_logger = pl_loggers.TensorBoardLogger(os.path.join(args.default_root_dir, 'tb_logs'))
    #학습률 기록
    lr_logger = pl.callbacks.LearningRateMonitor()
    # 트레이너 설정
    trainer = pl.Trainer.from_argparse_args(args, logger=tb_logger,
                                            callbacks=[checkpoint_callback, lr_logger], profiler="pytorch")
  # 모델 설정
    model = KoBARTConditionalGeneration(args, trainer)
    trainer.fit(model, dm )  
    # trainer.fit(model, dm ckpt_path="/content/drive/MyDrive/Github/KoBART-summarization/data/logs/last.ckpt")   #체크포인트 로드

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=None, amp_backend='native', amp_level=None, auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, batch_size=8, benchmark=None, check_val_every_n_epoch=1, checkpoint_path=None, default_root_dir='logs', detect_anomaly=False, deterministic=None, devices=None, enable_checkpointing=True, enable_model_summary=True, enable_progress_bar=True, fast_dev_run=False, gpus=1, gradient_clip_algorithm=None, gradient_clip_val=1.0, ipus=None, limit_predict_batches=None, limit_test_batches=None, limit_train_batches=None, limit_val_batches=None, log_every_n_steps=50, logger=True, lr=3e-05, max_epochs=10, max_len=512, max_steps=-1, max_time=None, min_epochs=None, min_steps=None, model_path=None, move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1, num_processes=None, num_sanity_val_steps=2, num_workers=4, overfit_batches=0.0, plugins=None, precision=32, profiler=None, reload_dataloaders_every_n_

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 103: 'val_loss' reached 1.80270 (best 1.80270), saving model to '/content/drive/MyDrive/Github/KoBART-summarization/logs/model_chp/epoch=00-val_loss=1.803.ckpt' as top 3


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 824: 'val_loss' was not in top 3


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 927: 'val_loss' was not in top 3


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 1030: 'val_loss' was not in top 3
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.profilers.profiler:FIT Profiler Report
Profile stats for: records
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::copy_         1.30%      11.695ms       126.35%        1.135s       6.600ms  

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
#모델 transformers 형태로 저장
!python get_model_binary.py --hparams /content/drive/MyDrive/Github/KoBART-summarization/logs/tb_logs/lightning_logs/version_0/hparams.yaml --model_binary /content/drive/MyDrive/Github/KoBART-summarization/logs/model_chp/epoch=02-val_loss=1.632.ckpt

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
def load_model():
    model = BartForConditionalGeneration.from_pretrained('./kobart_summary')
    return model


model = load_model()
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
text = """
기본세트 구매해서 엄마랑 하나씩 사용하고 있어요~너무나 유명한 제품이라 엄마가 받으시면 너무 좋아하셨어요ㅎㅎ아이크림 같은 다른 화장품으로 관리하기 힘든 부위의 주름을 손쉽게 관리할 수 있어서 좋구요, 전 개인적으로 목에 깊은 주름이 있어서 주로 사용합니다.요즘은 마스크 때문에 자극이 많은데 사용하기 좋습니다.보습력 좋고 향도 좋아요!!다만 별을 하나 뺀 이유는 회사에서 에어컨 바람에 얼굴이 건조해서 발랐는데, 더운 날씨라 그런지 몰라도흡수는 평소보다 더 안되고 번들번들해서 머리카락 다 들러붙어서 떡지더라구요ㅠ피부에 흡수되는 시간은 개인차가 다 있으시겠지만,저는 시간이 좀 걸려서 집에서만 사용합니다.연예인 분들처럼 바르고 나서 바로 뽀송하지는  않네요ㅎ
"""

text = text.replace('\n', ' ')
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'엄마랑 구매해서 엄마랑 하나씩 사용하고 있어요. 보습력 좋고 향도 좋아요. 별을 하나 빼고 흡수는 평소보다 더 안되고 번들번들해서 머리카락이 들러붙어서 떡지더라구요. 흡수는 시간이 좀 걸려서 집에서만 사용합니다.'

In [ ]:
#체크포인트 삭제
from glob import glob
[os.remove(f) for f in glob("/content/drive/MyDrive/Github/KoBART-summarization/logs/model_chp/*.ckpt")]

[None, None, None, None, None, None, None, None, None]

In [ ]:
#huggingface에 업로드
# Repository 생성 & model upload
REPO_NAME = "koBART_Sum_Review_finetuning" # ex) 'my-bert-fine-tuned'
AUTH_TOKEN =  # huggingface token
 
## Upload to Huggingface Hub
model.push_to_hub(
    REPO_NAME, 
    use_temp_dir=True, 
    use_auth_token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME, 
    use_temp_dir=True, 
    use_auth_token=AUTH_TOKEN
)